<a href="https://colab.research.google.com/github/pvfalcao/A-semi-supervised-learning-approach-with-data-drift-detection-to-deal-with-ransomware/blob/main/20s/Boostrapped_tree_ensemble_20S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importação de bibliotecas


In [ ]:
import time
import psutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import sklearn
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix


Recebendo conjunto de dados


In [ ]:
df = pd.read_csv(r'/content/drive/MyDrive/Navarra/N20S20/N20S20.csv', header = None)
zero = pd.read_csv(r'/content/drive/MyDrive/Navarra/N20S20/zeroDays.csv', header = None)
user = pd.read_csv(r'/content/drive/MyDrive/Navarra/N20S20/userSamples_train.csv', header = None)

targets = df.iloc[:,df.shape[1] - 1]
df = df.iloc[:,0:-1]
tz = zero.iloc[:,zero.shape[1] - 1]
zero = zero.iloc[:,0:-1]
tu = user.iloc[:,user.shape[1] - 1]
user = user.iloc[:,0:-1]


In [ ]:

#Initialize the variables

precision_list = []
recall_list = []
f1_list = []
accuracy_list = []


# Initialize 10-Fold

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
clf = RandomForestClassifier(
        n_estimators=10,            # number_of_models = 10
        bootstrap=True,                   # bootstrap sampling
        max_leaf_nodes=512,    # 512-node limit
        max_features=32,    # split_candidates = 32
        max_samples=1.0,          # sample_rate = 1.0
        random_state=42,         # deterministic behavior
        n_jobs=-1,                         # use all CPU cores
        ccp_alpha=0.001
        )


#Consumption metrics

psutil.cpu_percent(interval=None)
start = time.process_time()

# Split the dataset into 10 folds

for fold, (train_idx, test_idx) in enumerate(skf.split(df, targets)):
    # Select data using .iloc to get pandas DataFrames/Series
    train_features = df.iloc[train_idx]
    train_labels = targets.iloc[train_idx]
    test_features = df.iloc[test_idx]
    test_labels = targets.iloc[test_idx]
    TP = 0
    TN = 0
    FN = 0
    FP = 0


    X_train, X_test = df.iloc[train_idx], df.iloc[test_idx]
    y_train, y_test = targets.iloc[train_idx], targets.iloc[test_idx]


    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    TN, FP, FN, TP = confusion_matrix(y_pred, y_test).ravel()

    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    accuracy = (TN + TP)/(TP + FP + FN + TN)


    accuracy_list.append(accuracy)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_list.append(f1)

#Consumption measures
end = time.process_time()
print('The processing time was:', end - start)
cpu_usage_system = psutil.cpu_percent(interval=None)
print(f"System-wide CPU usage: {cpu_usage_system}%")
print(f"Load average:{psutil.getloadavg()}")
print(f"CPU Frequency:{psutil.cpu_freq()}")
print("CPU usage (%):", psutil.cpu_percent())
print("CPU usage (%):", psutil.cpu_percent())
ram = psutil.virtual_memory()
print("RAM usage (%):", ram.percent)
print("RAM used (GB):", round(ram.used / 1e9, 2))

#Metrics

mean_precision = sum(precision_list) / len(precision_list)
mean_recall = sum(recall_list) / len(recall_list)
mean_f1 = sum(f1_list) / len(f1_list)
mean_accuracy = sum(accuracy_list) / len(accuracy_list)

print(f'Mean precision: {mean_precision:.5f}')
print(f'Mean recall: {mean_recall:.5f}')
print(f'Mean F1 Score: {mean_f1:.5f}')
print(f'Mean accuracy: {mean_accuracy:.5f}')





The processing time was: 32.031356065000004
System-wide CPU usage: 98.2%
Load average:(2.08544921875, 0.82470703125, 0.31787109375)
CPU Frequency:scpufreq(current=2199.998, min=0.0, max=0.0)
CPU usage (%): 0.0
CPU usage (%): 0.0
RAM usage (%): 11.4
RAM used (GB): 1.21
Mean precision: 0.99392
Mean recall: 0.99585
Mean F1 Score: 0.99488
Mean accuracy: 0.99688


Teste Zero day

In [ ]:
TP = 0
FP = 0
FN = 0
TN = 0


y_pred = clf.predict(zero)

for i in range(tz.shape[0]):
       if y_pred[i] == 0 and tz[i] == 0:
          TN += 1
       elif y_pred[i] == 0 and tz[i] == 1:
          FN += 1
       elif y_pred[i] == 1 and tz[i] == 0:
          FP += 1
       elif y_pred[i] == 1 and tz[i] == 1:
          TP += 1

precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
accuracy = (TN + TP)/(TP + FP + FN + TN)

print(f'Precision: {precision:.5f}')
print(f'Recall: {recall:.5f}')
print(f'F1: {f1:.5f}')
print(f'Accuracy: {accuracy:.5f}')


Precision: 1.00000
Recall: 1.00000
F1: 1.00000
Accuracy: 1.00000


Teste amostras benignas

In [ ]:

TP = 0
FP = 0
FN = 0
TN = 0


y_pred = clf.predict(user)

for i in range(tu.shape[0]):
       if y_pred[i] == 0 and tu[i] == 0:
          TN += 1
       elif y_pred[i] == 0 and tu[i] == 1:
          FN += 1
       elif y_pred[i] == 1 and tu[i] == 0:
          FP += 1
       elif y_pred[i] == 1 and tu[i] == 1:
          TP += 1

precision = TP / (TP + FP) if TP + FP > 0 else 0
recall = TP / (TP + FN) if TP + FN > 0 else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
accuracy = (TN + TP)/(TP + FP + FN + TN)

print(f'Precision: {precision:.5f}')
print(f'Recall: {recall:.5f}')
print(f'F1: {f1:.5f}')
print(f'Accuracy: {accuracy:.5f}')


Precision: 0.00000
Recall: 0.00000
F1: 0.00000
Accuracy: 0.00000
